In [1]:
import sys
import os

# find and append the root directory
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))  # or ".." if in notebooks/
sys.path.append(project_root)

import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import time

from src.problems.benchmarks import (
    rosenbrock, rastrigin, ackley,
    grad_rosenbrock, grad_rastrigin, grad_ackley
)
from src.optimizers.sa import sa_continuous
from src.optimizers.gd import gradient_descent
from src.utils.utils_experiments import bootstrap_experiment

# Paths
results_dir = os.path.join(project_root, "results", "gridsearch")
os.makedirs(results_dir, exist_ok=True)

In [2]:
benchmarks = {
    "rosenbrock": (rosenbrock, grad_rosenbrock),
    "rastrigin": (rastrigin, grad_rastrigin),
    "ackley": (ackley, grad_ackley)
}

In [3]:
# hyperparameter grids for gridsearch
sa_grid = {
    'T0': [10, 50, 100],
    'alpha': [0.9, 0.95, 0.99, 0.995, 0.999],
    'step_size': [0.005, 0.01, 0.05, 0.1, 0.3, 0.5]
}

gd_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.3, 0.5]
}

In [4]:
# SA gridsearch helpers
def run_one_sa(f, T0, alpha, step_size, num_runs=20):
    result = bootstrap_experiment(
        sa_continuous, num_runs, f,
        x_init=None, bounds=[(-5, 5), (-5, 5)],
        T0=T0, alpha=alpha, step_size=step_size,
        tol=1e-6, max_iter=20000
    )
    stats = result["stats"]
    return {
        "T0": T0, "alpha": alpha, "step_size": step_size,
        "mean": stats["mean"], "best": stats["best"], "worst": stats["worst"], "std": stats["std"],
        "epsilon": stats["epsilon"], "near_optimal_count": stats["near_optimal_count"]
    }

def grid_search_sa(f, grid, num_runs=20, benchmark_name=""):
    combos = list(itertools.product(grid["T0"], grid["alpha"], grid["step_size"]))
    total = len(combos)
    results = []

    for idx, (T0, alpha, step_size) in enumerate(combos, 1):
        print(f"[SA | {benchmark_name}] Combo {idx}/{total} — Running...")
        res = run_one_sa(f, T0, alpha, step_size, num_runs)
        results.append(res)

    return pd.DataFrame(results)

In [8]:
# GD grid search helpers
def run_one_gd(f, grad, lr, num_runs=2):
    result = bootstrap_experiment(
        gradient_descent, num_runs, f, grad,
        lr=lr, tol=1e-6, max_iter=20000, x_init=None
    )
    stats = result["stats"]
    return {
        "lr": lr,
        "mean": stats["mean"], "best": stats["best"], "worst": stats["worst"], "std": stats["std"],
        "epsilon": stats["epsilon"], "near_optimal_count": stats["near_optimal_count"]
    }

def grid_search_gd(f, grad, grid, num_runs=2, benchmark_name=""):
    combos = list(grid["lr"])
    total = len(combos)
    results = []

    for idx, lr in enumerate(combos, 1):
        print(f"[GD | {benchmark_name}] Combo {idx}/{total} — Running...")
        res = run_one_gd(f, grad, lr, num_runs)
        results.append(res)

    return pd.DataFrame(results)

# Running actual gridsearch

In [9]:
# run the grid search and save results
for name, (f, grad) in benchmarks.items():
    print(f"\nBenchmark: {name}")

    print("Running GD grid search")
    df_gd = grid_search_gd(f, grad, gd_grid, num_runs=20, benchmark_name=name)
    df_gd.to_csv(os.path.join(results_dir, f"gridsearch_gd_{name}.csv"), index=False)

    print("Running SA grid search")
    df_sa = grid_search_sa(f, sa_grid, num_runs=20, benchmark_name=name)
    df_sa.to_csv(os.path.join(results_dir, f"gridsearch_sa_{name}.csv"), index=False)


Benchmark: rosenbrock
Running GD grid search
[GD | rosenbrock] Combo 1/7 — Running...
Run 1/20...
Clip range : (-5, 5)
Run 2/20...
Clip range : (-5, 5)
Run 3/20...
Clip range : (-5, 5)
Run 4/20...
Clip range : (-5, 5)
Run 5/20...
Clip range : (-5, 5)
Run 6/20...
Clip range : (-5, 5)
Run 7/20...
Clip range : (-5, 5)
Run 8/20...
Clip range : (-5, 5)
Run 9/20...
Clip range : (-5, 5)
Run 10/20...
Clip range : (-5, 5)
Run 11/20...
Clip range : (-5, 5)
Run 12/20...
Clip range : (-5, 5)
Run 13/20...
Clip range : (-5, 5)
Run 14/20...
Clip range : (-5, 5)
Run 15/20...
Clip range : (-5, 5)
Run 16/20...
Clip range : (-5, 5)
Run 17/20...
Clip range : (-5, 5)
Run 18/20...
Clip range : (-5, 5)
Run 19/20...
Clip range : (-5, 5)
Run 20/20...
Clip range : (-5, 5)
[GD | rosenbrock] Combo 2/7 — Running...
Run 1/20...
Clip range : (-5, 5)
Run 2/20...
Clip range : (-5, 5)
Run 3/20...
Clip range : (-5, 5)


KeyboardInterrupt: 